# Retrieval 

@Authors: [Franck Porteous](https://github.com/FranckPrts)  [Faith Hunja](https://github.com/faithhunja), [Hannah Krohn](https://github.com/hlili303)


This page is used to import the data from ECCO and UHSLC. All ECCO data from 1992-2017 is imported and longest available data gauge data from locations of interest are also included, namely five tidal gauges in Puerto Rico including Isabel Segunda, Esperanza, Arecibo, Mayaguez, and Fajardo. 

## Setup

### Imports

In [1]:
# imports

import matplotlib.pyplot as plt
import xarray as xr
import os
import pooch
import tempfile
from pooch import HTTPDownloader
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
from cartopy import crs as ccrs
from scipy import stats

### Important variables

In [2]:
storm_repo_full = {
    "Sandy": {
        "start_date": datetime(1992, 1, 1),
        "end_date": datetime(2017, 12, 31),
        "tide": "Spring",
        "duration": 9497,
        "maxIntensity_kt": 100,
        "maxIntensity_mb": 940,
        "tg": {"Atlantic City, NJ": {"lat": 39.35500, "lon": -74.41800},
               "Montauk, NY": {"lat": 40.693, "lon": -72.049},
               "The Battery, NY": {"lat": 40.701, "lon": -74.014}},
        "tg_precise": {
            "Atlantic City, NJ (precise)": {"lat": 39.35500, "lon": -74.41800, 
                                            "record_start": datetime(1992, 1, 1), 
                                            "record_end": datetime(2017, 12, 31)},
            "Montauk, NY (precise)": {"lat": 40.693, "lon": -72.049, 
                                      "record_start": datetime(1992, 1, 1), 
                                      "record_end": datetime(2017, 12, 31)},
            "The Battery, NY (precise)": {"lat": 40.701, "lon": -73.75, 
                                          "record_start": datetime(1992, 1, 1), 
                                          "record_end": datetime(2017, 12, 31)}
        }
    },

    "Maria": {
        "start_date": datetime(1992, 1, 1),
        "end_date": datetime(2017, 12, 31),
        "tide": "Spring",
        "duration": 9497,
        "maxIntensity_kt": 150,
        "maxIntensity_mb": 908,
        "tg": {"Penuelas, PR": {"lat": 17.972, "lon": -66.762},
               "Isabel Segunda, PR": {"lat": 18.152, "lon": -65.443},
               "Esperanza, PR": {"lat": 18.093, "lon": -65.47},
               "Arecibo, PR": {"lat": 18.48, "lon": -66.702},
               "Mayaguez, PR": {"lat": 18.22, "lon": -67.16},
               "Fajardo, PR": {"lat": 18.333, "lon": -65.633}},
        "tg_precise": {
            "Penuelas, PR (precise)": {"lat": 17.4, "lon": -66.762, 
                                       "record_start": datetime(2001, 4, 1), 
                                       "record_end": datetime(2005, 2, 9)},
            "Isabel Segunda, PR (precise)": {"lat": 18.152, "lon": -65.443, 
                                             "record_start": datetime(2009, 3, 7), 
                                             "record_end": datetime(2017, 10, 19)},
            "Esperanza, PR (precise)": {"lat": 18.093, "lon": -65.47, 
                                        "record_start": datetime(2005, 8, 16), 
                                        "record_end": datetime(2017, 12, 31)},
            "Arecibo, PR (precise)": {"lat": 18.5, "lon": -66.702, 
                                      "record_start": datetime(2008, 8, 29), 
                                      "record_end": datetime(2017, 12, 31)},
            "Mayaguez, PR (precise)": {"lat": 18.22, "lon": -67.16, 
                                       "record_start": datetime(2008, 3, 11), 
                                       "record_end": datetime(2017, 12, 31)},
            "Fajardo, PR (precise)": {"lat": 18.333, "lon": -65.633, 
                                      "record_start": datetime(1992, 1, 1), 
                                      "record_end": datetime(2017, 12, 31)}
        }
    },

    "Ketsana": {
        "start_date": datetime(1992, 1, 1),
        "end_date": datetime(2017, 12, 31),
        "tide": "Neap",
        "duration": 9497,
        "maxIntensity_kt": 90,
        "maxIntensity_mb": 955,
        "tg": {"Malakal, Palau": {"lat": 7.33000, "lon": 134.46300},
               "Legaspi, Philippines": {"lat": 13.15000, "lon": 123.75000},
               "Manila, Philippines": {"lat": 14.58500 , "lon": 120.96800},
               "Subic Bay, Philippines": {"lat": 14.76500 , "lon": 120.25200},
               "Qui Nhon, Vietnam": {"lat": 13.77500, "lon": 109.25500}},
        "tg_precise": {
            "Malakal, Palau (precise)": {"lat": 7.33000, "lon": 134.46300, 
                                         "record_start": datetime(1992, 1, 1), 
                                         "record_end": datetime(2017, 12, 31)},
            "Legaspi, Philippines (precise)": {"lat": 13.15000, "lon": 124.1, 
                                               "record_start": datetime(1992, 1, 1), 
                                               "record_end": datetime(2017, 9, 5)},
            "Subic Bay, Philippines (precise)": {"lat": 14.76500 , "lon": 119.9, 
                                                 "record_start": datetime(2007, 2, 27), 
                                                 "record_end": datetime(2017, 12, 31)},
            "Qui Nhon, Vietnam (precise)": {"lat": 13.77500, "lon": 109.25500, 
                                            "record_start": datetime(2007, 10, 19), 
                                            "record_end": datetime(2017, 12, 31)}
        }
    }
}

storm_repo_event = {    
             "Sandy":{"start_date": datetime(2012, 10, 21), 
                      "end_date": datetime(2012, 10, 31),
                      "tide": "Spring",
                      "duration": 10,
                      "maxIntensity_kt": 100,
                      "maxIntensity_mb": 940,
                      "tg": {"Atlantic City, NJ": {"lat": 39.35500, "lon": -74.41800},
                            "Montauk, NY": {"lat": 40.693, "lon": -72.049},
                            "The Battery, NY": {"lat": 40.701, "lon": -74.014}},
                      
                      "tg_precise": {"Atlantic City, NJ (precise)": {"lat": 39.35500, "lon": -74.41800},
                            "Montauk, NY (precise)": {"lat": 40.693, "lon": -72.049},
                            "The Battery, NY (precise)": {"lat": 40.701, "lon": -73.75}}
                    },
              
             "Maria":{"start_date": datetime(2017, 9, 16), 
                      "end_date": datetime(2017, 10, 2),
                      "tide": "Spring",
                      "duration": 17,
                      "maxIntensity_kt": 150,
                      "maxIntensity_mb": 908,
                      "tg": {"Penuelas, PR": {"lat": 17.972, "lon": -66.762},
                            "Isabel Segunda, PR": {"lat": 18.152, "lon": -65.443},
                            "Esperanza, PR": {"lat": 18.093, "lon": -65.47},
                            "Arecibo, PR": {"lat": 18.48, "lon": -66.702},
                            "Mayaguez, PR": {"lat": 18.22, "lon": -67.16},
                            "Fajardo, PR": {"lat": 18.333, "lon": -65.633}},

                      "tg_precise": {"Penuelas, PR (precise)": {"lat": 17.4, "lon": -66.762},
                            "Isabel Segunda, PR (precise)": {"lat": 18.152, "lon": -65.443},
                            "Esperanza, PR (precise)": {"lat": 18.093, "lon": -65.47},
                            "Arecibo, PR (precise)": {"lat": 18.5, "lon": -66.702},
                            "Mayaguez, PR (precise)": {"lat": 18.22, "lon": -67.16},
                            "Fajardo, PR (precise)": {"lat": 18.333, "lon": -65.633}}
                    }, 
              
             "Ketsana":{"start_date": datetime(2009, 9, 25), 
                      "end_date": datetime(2009, 9, 30),
                      "tide": "Neap",
                      "duration": 5,
                      "maxIntensity_kt": 90,
                      "maxIntensity_mb": 955,
                      "tg": {"Malakal, Palau": {"lat": 7.33000, "lon": 134.46300},
                            "Legaspi, Philippines": {"lat": 13.15000, "lon": 123.75000},
                            "Manila, Philippines": {"lat": 14.58500 , "lon": 120.96800},
                            "Subic Bay, Philippines": {"lat": 14.76500 , "lon": 120.25200},
                            "Qui Nhon, Vietnam": {"lat": 13.77500, "lon": 109.25500}},
                      "tg_precise": {"Malakal, Palau (precise)": {"lat": 7.33000, "lon": 134.46300},
                            "Legaspi, Philippines (precise)": {"lat": 13.15000, "lon": 124.1},
                  #          "Manila, Philippines (precise)": {"lat": 14.58500 , "lon": 120.96800},
                            "Subic Bay, Philippines (precise)": {"lat": 14.76500 , "lon": 119.9},
                            "Qui Nhon, Vietnam (precise)": {"lat": 13.77500, "lon": 109.25500}}
                    }
             }
tg_repo = {"Sandy": {"Atlantic City, NJ": "https://uhslc.soest.hawaii.edu/data/netcdf/fast/hourly/h264.nc"},
           "Maria": {"Penuelas, PR": "https://uhslc.soest.hawaii.edu/data/netcdf/rqds/atlantic/hourly/h243a.nc",
                    "Isabel Segunda, PR": "https://uhslc.soest.hawaii.edu/data/netcdf/rqds/atlantic/hourly/h732a.nc",
                     "Esperanza, PR":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/atlantic/hourly/h733a.nc",
                     "Arecibo, PR":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/atlantic/hourly/h735a.nc",
                     "Mayaguez, PR":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/atlantic/hourly/h736a.nc",
                     "Fajardo, PR":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/atlantic/hourly/h783b.nc"},
           "Ketsana": {"Malakal, Palau":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/pacific/hourly/h007b.nc",
                      "Legaspi, Philippines":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/pacific/hourly/h371a.nc",
                      #"Manila, Philippines":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/pacific/hourly/h370a.nc",
                      "Subic Bay, Philippines":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/pacific/hourly/h382a.nc",
                      "Qui Nhon, Vietnam":"https://uhslc.soest.hawaii.edu/data/netcdf/rqds/pacific/hourly/h381b.nc"}
          }

### Helper function

The function `get_ds_for_date_range` returns a list of the ECCO dataset for a given time range.
***Don't forget to make an account [here](https://urs.earthdata.nasa.gov/home)***

In [3]:
auth=('username', 'password')

In [4]:
def get_ds_for_date_range(start_date, end_date, lati, long, auth=auth):
    
    # Calculate the number of days in the range
    num_days = (end_date - start_date).days + 1
        
    print("range", start_date, end_date)

    datasets = []
    ds_files = []

    for i in range(num_days):
        date_req = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
        
        #ecco_url = "https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/ECCO_L4_SSH_05DEG_DAILY_V4R4/"
        ecco_url = "https://ecco.jpl.nasa.gov/drive/files/Version4/Release4b/latlon/daily/SSH"

        #file_req = f"SEA_SURFACE_HEIGHT_day_mean_{date_req}_ECCO_V4r4_latlon_0p50deg.nc"
        file_req = f"SEA_SURFACE_HEIGHT_day_mean_{date_req}_ECCO_V4r4b_latlon_0p50deg.nc"
        
        if not os.path.exists(f"./data/{file_req}"):
            print(f"downloading: {date_req}")
            
            fn = os.path.join(ecco_url, file_req)

            # let the downloader know the login credentials
            downloader = HTTPDownloader(auth=auth)
            downloader(url=fn, output_file="./data/{}".format(file_req), pooch=None)

            dataset = xr.open_dataset("./data/{}".format(file_req))
            
            dataset.sel(
                latitude  = lati,
                longitude = long,
                method='nearest')
            
            datasets.append(dataset)
            ds_files.append(file_req)

        else:
            try:
                dataset = xr.open_dataset("./data/{}".format(file_req))
                #print(f"checked ./data/{file_req} succesfully")
            except:    
                print(f'Cannot process: {file_req}')
                break
            
            
            dataset.sel(
                latitude  = lati,
                longitude = long,
                method='nearest')
            
            datasets.append(dataset)
            ds_files.append(file_req)
    
    return datasets, ds_files


## Getting the data

In [5]:
event_to_process = "Maria"
locations_to_process = list(storm_repo_full[event_to_process]["tg_precise"].keys())
print(locations_to_process)

['Penuelas, PR (precise)', 'Isabel Segunda, PR (precise)', 'Esperanza, PR (precise)', 'Arecibo, PR (precise)', 'Mayaguez, PR (precise)', 'Fajardo, PR (precise)']


### ECCO data 

ECCO data takkes time to access and is resource intensive. To aleviate these issues, we demonstrate how to download the ECCO data localy to store it in a `xr.Dataset` format.

Because we want this code to remain flexible, we did not make a loop to get all the location's data. Instead we provide a code where this can be done by manually selecting name of a location. Here we demonstrate the process for the location `'Isabel Segunda, PR (precise)'`. 

In [6]:
# Set up where data is being stored
ds_allLoc_allEvent_sat = {}
ar_allLoc_allEvent_sat = {}

If we want to have more than one location, we would have to repeat the steps below for each of them

In [7]:
# Chose a location
location_to_process = locations_to_process[1]

# Print the date that will be used 
print(f"event: {location_to_process}\nstart_date: {storm_repo_full[event_to_process]['tg_precise'][location_to_process]['record_start']}\nend date: {storm_repo_full[event_to_process]['tg_precise'][location_to_process]['record_end']}")

event: Isabel Segunda, PR (precise)
start_date: 2009-03-07 00:00:00
end date: 2017-10-19 00:00:00


In [8]:
print(f'getting the data for {location_to_process} ({event_to_process})')

ds_loc = {}
ar_loc = {}

start_date = storm_repo_full[event_to_process]["tg_precise"][location_to_process]["record_start"]
end_date = storm_repo_full[event_to_process]["tg_precise"][location_to_process]["record_end"]

ds, filename = get_ds_for_date_range(start_date, 
                                     end_date, 
                                     lati=storm_repo_full[event_to_process]["tg_precise"][location_to_process]["lat"],
                                     long=storm_repo_full[event_to_process]["tg_precise"][location_to_process]["lon"]
                                    )

print(f'done getting the data for {location_to_process} ({event_to_process})')


getting the data for Isabel Segunda, PR (precise) (Maria)
range 2009-03-07 00:00:00 2017-10-19 00:00:00
done getting the data for Isabel Segunda, PR (precise) (Maria)


In [9]:
# Now we select the CLOSEST gridcell that correspond to the location (hence the suffix (precise))
ds_sel = [d.sel(latitude=storm_repo_full[event_to_process]["tg_precise"][f"{location_to_process}"]["lat"],
                 longitude=storm_repo_full[event_to_process]["tg_precise"][f"{location_to_process}"]["lon"],
                 method='nearest') for d in ds]

In [10]:
# We now concatenate the list of dataset in one 
ds_to_save = xr.concat(ds_sel, dim='time')
print(f'done concat for {location_to_process[:-10]} ({event_to_process})')

done concat for Isabel Segunda, PR (Maria)


In [12]:
# Save the array
np.save(f"./isp_saved_arrays/ECCO_{event_to_process}-{location_to_process[:-10]}.npy", ds_to_save["SSH"].values)

# Save rthe d
ds_to_save.to_netcdf(path=f"./isp_saved_ds/ECCO_{location_to_process[:-10]}.nc", mode='w')

### Tidal gauge data 

Because the tidal gauge data is fast to get, we store the array and datasets of all location all at once.

In [14]:
# We set resample to true to get daily data instead of hourly data 
resample_tg    = True

In [18]:
ar_allLoc_allEvent_tg = {} #array
ds_allLoc_allEvent_tg = {} #dataset

# Setting up empty dicts for each event outside the loop
ar_allLoc_allEvent_tg[event_to_process] = {}
ds_allLoc_allEvent_tg[event_to_process] = {}

for lo in locations_to_process: 
    print(f'doing: {event_to_process} - {lo}')

    locations = tg_repo.get(event_to_process)

    # Get URL
    url_choosen = locations[lo[:-10]]

    # Retrieve dataset 
    ds = xr.open_dataset(pooch.retrieve(url_choosen, known_hash=None))

    # Cut the time of the event from ds
    start_date = storm_repo_full[event_to_process]["tg_precise"][f'{lo}']["record_start"]
    end_date = storm_repo_full[event_to_process]["tg_precise"][f'{lo}']["record_end"]

    # Get the event data
    ds_event = ds.sel(time=slice(start_date, end_date))

    if resample_tg:
      
        try:
            resampled = ds_event.resample(time="D")
            resampled = resampled.mean()
            
            # Save the array and divide by 1000 for unit conversion (millimeter to M)
            np.save(f"./isp_saved_arrays/TG_{event_to_process}-{lo[:-10]}.npy", resampled.sea_level.values.flatten() / 1000)
            
            # Save the dataset
            resampled.to_netcdf(path=f"./isp_saved_ds/TG_{lo[:-10]}.nc", mode='w') 
            print("Done with resample_tg")
        
        except:
            print("couldn't do resample_tg")
            pass
    else:
        # Save the array and divide by 1000 for unit conversion (millimeter to M)
        np.save(f"./isp_saved_arrays/TG_{event_to_process}-{lo[:-10]}_NotRsampled.npy", ds_event.sea_level.values.flatten() / 1000)
        
        # Save the dataset
        ds_event.to_netcdf(path=f"./isp_saved_ds/TG_{lo[:-10]}.nc", mode='w') 
        
print('> Done saving TG datasets and arrays')

doing: Maria - Penuelas, PR (precise)
Done with resample_tg
doing: Maria - Isabel Segunda, PR (precise)
Done with resample_tg
doing: Maria - Esperanza, PR (precise)
Done with resample_tg
doing: Maria - Arecibo, PR (precise)
Done with resample_tg
doing: Maria - Mayaguez, PR (precise)
Done with resample_tg
doing: Maria - Fajardo, PR (precise)
Done with resample_tg
> Done saving TG datasets and arrays
